In [47]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import ppscore as pps
import statsmodels.formula.api as sm
import xgboost as xgb

from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split, RandomizedSearchCV, StratifiedKFold
from scipy.stats import mode

from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.compose import ColumnTransformer


np.set_printoptions(threshold=np.inf)

In [48]:
train = pd.read_csv('Train.csv')
test = pd.read_csv('Test.csv')

df = pd.concat([train, test])
len(train)

8523

In [49]:
df = pd.read_csv('Train.csv')

In [50]:
piv = pd.pivot_table(df,  columns=['Outlet_Type'], values=['Outlet_Size'], aggfunc=(lambda x:mode(x)[0]) )
missing_rows = df['Outlet_Size'].isnull() 
df.loc[missing_rows,'Outlet_Size'] = df.loc[missing_rows,'Outlet_Type'].apply(lambda x: piv[x])

In [51]:
piv_weight = df.pivot_table(values = 'Item_Weight', columns='Item_Identifier')

In [52]:
total_weight_mean = df['Item_Weight'].dropna().values.mean()
total_weight_mean

12.857645184135977

In [53]:
for i in df['Item_Identifier'].values:
    if i not in piv_weight.columns:
        print(i)

FDN52
FDK57
FDE52
FDQ60


In [54]:
df.loc[(df['Item_Identifier'] == 'FDN52')|(df['Item_Identifier'] == 'FDE52')|(df['Item_Identifier'] == 'FDK57')|(df['Item_Identifier'] == 'FDQ60'), 'Item_Weight'] = total_weight_mean

In [55]:
missing_rows = df['Item_Weight'].isnull()

In [56]:
df.loc[missing_rows, 'Item_Weight'] = df.loc[missing_rows, 'Item_Identifier'].apply(lambda row: piv_weight[row])

In [57]:
fat_content_dict = {'LF': 'LF', 'low fat': 'LF', 'Low Fat': 'LF', 'reg': 'R', 'Regular': 'R'}

In [58]:
df.loc[:, 'Item_Fat_Content'] = df.loc[:, 'Item_Fat_Content'].apply(lambda row: fat_content_dict[row])

In [59]:
class_dict = {'FD': 'Food Items', 'DR': 'Drinks', 'NC': 'Non Consumables'}
df['Item_Class'] = df['Item_Identifier'].apply(lambda row: row[:2]).map(class_dict)

In [60]:
df.drop(columns=['Item_Identifier', 'Outlet_Identifier'], inplace = True)

In [61]:
categ_cols = ['Item_Fat_Content','Item_Type', 'Outlet_Size', 'Outlet_Location_Type', 'Outlet_Type', 'Item_Class']
num_col = ['Item_Weight', 'Item_Visibility', 'Item_MRP' ,'Outlet_Establishment_Year', 'Item_Outlet_Sales']

cat_df = df[categ_cols]
num_df = df[num_col]

transformer = ColumnTransformer(transformers=[("OneHot",OneHotEncoder(),categ_cols)],remainder='passthrough')
cat_df = transformer.fit_transform(cat_df).toarray()

mmscaler = MinMaxScaler()
num_df = mmscaler.fit_transform(num_df)

In [62]:
X = np.append(num_df, cat_df, axis = 1)

In [63]:
y = df.iloc[:, -2:-1].values

In [64]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state = 0 )

In [65]:
grid = {
    'eta': .1,
    'gamma': 0,
    'max_depth': 4,
    'subsample': 0.5,
    'reg_lambda': 0.1,
    'alpha': 0.5
}

In [66]:
xgbreg = XGBRegressor()

In [67]:
data_dmatrix = xgb.DMatrix(data=X,label=y)

In [68]:
cv_results = xgb.cv(dtrain=data_dmatrix, params=grid, nfold=3,
                    num_boost_round=50,early_stopping_rounds=10,metrics="rmse", as_pandas=True, seed=0)